<a href="https://colab.research.google.com/github/trollmannen/Data-Science/blob/main/WoW-Auction-House-EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [24]:
CLIENT_ID = ""
CLIENT_SECRET = ""
REGION = "eu" #"us"
LOCALE = "en_GB" #"en_US"

In [3]:
#!pip install python-blizzardapi

In [4]:
from blizzardapi import BlizzardApi
api_client = BlizzardApi(CLIENT_ID, CLIENT_SECRET)

## Part 1: Gather & Store Data

In [5]:
import requests
def create_access_token():
  data = { 'grant_type': 'client_credentials' }
  response = requests.post(f'https://{REGION}.battle.net/oauth/token',
                           data=data, auth=(CLIENT_ID, CLIENT_SECRET))
  return response.json()['access_token']

access_token = create_access_token()

In [6]:
def get_connected_realms_index(access_token):
  connected_realms_index = {}
  response = requests.get(f'https://{REGION}.api.blizzard.com/data/wow/search/'
                          f'connected-realm?namespace=dynamic-{REGION}'
                          f'&access_token={access_token}')
  for connected_realm in response.json()['results']:
    for realm in connected_realm['data']['realms']:
      connected_realms_index[realm['slug']] = connected_realm['data']['id']

  return pd.Series(connected_realms_index)

connected_realms_index = get_connected_realms_index(access_token)

In [7]:
def get_auctions(connected_realm_id):
  auction_id,item_id,pet_breed_id,pet_level,pet_quality_id,pet_species_id,unit_price,buyout,quantity,time_left = ([] for i in range(10))
  response = api_client.wow.game_data.get_auctions(REGION,
                                                   LOCALE,
                                                   connected_realm_id)
  for auction in response['auctions']:
    auction_id.append(auction['id'])    
    item_id.append(auction['item']['id'])

    if auction['item']['id'] == 82800:
      pet_breed_id.append(auction['item']['pet_breed_id'])
      pet_level.append(auction['item']['pet_level'])
      pet_quality_id.append(auction['item']['pet_quality_id'])
      pet_species_id.append(auction['item']['pet_species_id'])
    else:
      pet_breed_id.append(0)
      pet_level.append(0)
      pet_quality_id.append(0)
      pet_species_id.append(0)

    if 'unit_price' in auction:
      unit_price.append(auction['unit_price'])
    else:
      unit_price.append(0)

    if 'buyout' in auction:
      buyout.append(auction['buyout'])
    else:
      buyout.append(0)

    quantity.append(auction['quantity'])
    time_left.append(auction['time_left'])

  auctions = pd.DataFrame(list(zip(auction_id,item_id,pet_breed_id,pet_level,pet_quality_id,
                                   pet_species_id,unit_price,buyout,quantity,time_left)),
                          columns=['auction_id','item_id','pet_breed_id','pet_level','pet_quality_id',
                                   'pet_species_id','unit_price','buyout','quantity','time_left'])
  
  auctions['crid'] = connected_realm_id

  return auctions

In [8]:
all_auctions = pd.DataFrame(columns=['auction_id','item_id','pet_breed_id','pet_level','pet_quality_id',
                                     'pet_species_id','unit_price','buyout','quantity','time_left','crid'])

In [9]:
from tqdm import tqdm
for connected_realm_id in tqdm(connected_realms_index.unique()):
  all_auctions = pd.concat([get_auctions(connected_realm_id),
                            all_auctions], axis=0)


100%|██████████| 92/92 [03:11<00:00,  2.08s/it]


In [10]:
def get_item(item_id):
  item = api_client.wow.game_data.get_item(region=REGION,
                                           locale=LOCALE,
                                           item_id=item_id)
  return item['name']

#all_auctions['item'] = np.vectorize(get_item)(all_auctions['item_id'])

In [11]:
def get_pet(pet_species_id):
  if pet_species_id == 0:
    return pet_species_id
  pet = api_client.wow.game_data.get_pet(region=REGION,
                                         locale=LOCALE,
                                         pet_id=pet_species_id)
  return pet['name']

#all_auctions['pet_species_id'] = np.vectorize(get_pet)(all_auctions['pet_species_id'])

In [12]:
all_auctions.to_csv('all_auctions.csv',index=False)

## Part 2: Clean & Organize Data

In [13]:
all_auctions = pd.read_csv('all_auctions.csv',index_col='auction_id')

In [14]:
all_auctions['buyout'] = all_auctions['unit_price'] + all_auctions['buyout']

In [15]:
all_auctions = all_auctions[all_auctions['buyout'] > 0].drop('unit_price',axis=1)

In [16]:
# todo: fix this monster
all_auctions['gold'] = all_auctions['buyout'].astype(str).str[:-4]
all_auctions['silver'] = all_auctions['buyout'].astype(str).str[-4:-2]
all_auctions['copper'] = all_auctions['buyout'].astype(str).str[-2:]

all_auctions['gold'] = pd.to_numeric(all_auctions['gold'])
all_auctions['silver'] = pd.to_numeric(all_auctions['silver'])
all_auctions['copper'] = pd.to_numeric(all_auctions['copper'])

all_auctions['gold'] = all_auctions['gold'].fillna(0)
all_auctions['silver'] = all_auctions['silver'].fillna(0)
all_auctions['copper'] = all_auctions['copper'].fillna(0)

all_auctions['gold'] = all_auctions['gold'].astype('int')
all_auctions['silver'] = all_auctions['silver'].astype('int')
all_auctions['copper'] = all_auctions['copper'].astype('int')

## Part 3: EDA

### How many realms are included in all the retail EU Realms and this EDA?

In [23]:
len(connected_realms_index)

267

### What percentage of all auctions are battle pet auctions?

In [19]:
round(len(all_auctions[all_auctions['item_id']==82800])/len(all_auctions)*100,2)

8.94

### How many auctions for unique items are there?

In [21]:
all_auctions['item_id'].nunique()

21920